# Monitor-modelling

Modelling of monitor screen sizes and the distance you sit from them. A Jupyter notebook python program that uses adjustment of widgets to model different scenarios.

Using toggle buttons selection is made of one of four different resolution 16:9 High Definition monitors. Using slider controls adjustments are made to the screen size and the distance from the monitor. 


For example the following information is displayed when sitting 1 meter from a 28 inch Ultra High Definition monitor: 

```
    Horizontal Pixels: 1920
    Vertical Pixels: 1080
    Screen Size: 28in.
    Screen Size: 71.1cm
    Width of Screen: 62.0cm
    Height of Screen: 34.9cm
    Screen distance: 100cm
    Viewing Angle: 34.4°
    Pixels per degree: 55.7
    Distance to screen edge: 104.7cm
    Difference: Centre to Edge: 4.7cm
    Difference: Centre to Edge: 4.7%
```
There are many on-line guides that recommend optimum viewing angles and screen distances. This program may be assistive to use in conjunction with the recommendations. 

For example: 
https://www.rtings.com/tv/reviews/by-size/size-to-distance-relationship

## Note to Python programmers.

The Jupyter notebook code cell was developed on a Ubuntu-Mate platform as of Sep 2019:

With Ubuntu 18.04.3 LTS the apt repository loads Jupyter with the version of the notebook server: `5.2.2`, where `print(ipywidgets.version_info)` returns: `(6, 0, 0)`

With Jupyter installation via PYPI in a virtual environment then pip loads Jupyter with the version of the notebook server: `6.0.1`, where `print(ipywidgets.version_info)` returns: `(7, 5, 1, 'final', 0)` 

Be aware that there are some differences in widget behaviour between versions. Please refer to the changelog: https://ipywidgets.readthedocs.io/en/latest/changelog.html


In [19]:
import ipywidgets as widgets
import math

# monitor-modelling
# Using widgets model for various High Definition monitors the size and 
# the distance from the screen to the viewer.
# In text area observe changes to viewing angle, pixels per degree, etc.

# Author: Ian Stewart. 2019-10-03 
# Copyright: CC0

# References:
# https://en.m.wikipedia.org/wiki/16:9_aspect_ratio
# https://www.mathopenref.com/trigtangent.html
# https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html
# https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20List.html

# Issue with using "style" on Sliders with Version 6. OK in version 7.5.
# TraitError: The 'style' trait of an IntSlider instance must be a SliderStyle, 
# but a value of class 'dict' (i.e. {'description_width': 'initial'}) was specified.
# To obtain version uncomment...
# print(widgets.version_info)

#=== Constants

CM_PER_INCH = 2.54
ASPECT_RATIO = (16,9)

DATA_DESCRIPTION = [
    "Vertical Pixels",
    "Horizontal Pixels",
    "Screen Size",
    "Screen Size",
    "Tan of aspect ratio",
    "Adjacent Angle",
    "Atan of aspect ratio",
    "Width of Screen",
    "Height of Screen",
    "Half Screen Width",
    "Screen distance",
    "Half Tan Viewing Ratio",
    "Viewing Angle",
    "Pixels per degree",
    "Distance to screen edge",
    "Difference: Centre to Edge",
    "Difference: Centre to Edge",
]

DATA_UNIT = [
    "",
    "",
    "in.",
    "cm",
    "",
    "°",
    "",
    "cm",
    "cm",
    "cm",
    "cm",
    "",
    "°",
    "",
    "cm",
    "cm",
    "%",        
]

RELEVANT_DATA = [1, 0, 2, 3, 7, 8, 10, 12, 13, 14, 15, 16,]

#=== Instatiate Widgets

# Set style so labels description width is not truncated
style_1 = {'description_width': 'initial'}
style_2 = {'description_width': 'initial', 'font_weight': 'bold',}

# Set layout for text area
layout_1 = widgets.Layout(width='60%', height='300px',)

# Add WQHD 2560 x 1140?
toggle_button = widgets.ToggleButtons(
    options=['720', '1080', '2160', '4320'],
    description='16:9 Screen Heights:',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltips=['High Definition 1280 x 720', 
              'Full High Definition 1920 x 1080', 
              '4K Ultra High Definition 3840 x 2160', 
              '8K Ultra High Definition 7680 x 4320'],
    # icons=['check'] * 4,
    style=style_1,
)

# Adjusting Distance from centre of screen in meters
if widgets.version_info[0] < 7:  # (7, 5, 1, 'final', 0)
    slider_1 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=5.0,
            step=0.1,
            description='Screen Distance (m):',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            #style=style_1, #<-- Removed. Issue with version 6
        )
else:
    slider_1 = widgets.FloatSlider(
            value=0.5,
            min=0.5,
            max=5.0,
            step=0.1,
            description='Screen Distance (m):',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='.1f',
            style=style_1,
        )    

# Diagonal Screen Size in inches
if widgets.version_info[0] < 7: 
    slider_2 = widgets.IntSlider(
        value=12,
        min=12,
        max=100,
        step=1,
        description='Screen Size (in):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        #style=style_1, #<-- Removed. Issue with version 6
    )
else:
    slider_2 = widgets.IntSlider(
        value=12,
        min=12,
        max=100,
        step=1,
        description='Screen Size (in):',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        style=style_1,
    )    

text_area = widgets.Textarea(
    value='',
    placeholder='',
    description='Modelling Data:',
    disabled=False,
    layout=layout_1,
    style=style_2,   
)

#=== Handlers for widget changes. Call main(). Don't use passed values.
    
def toggle_button_handle_change(change):
    # {'name': 'value', 'old': '720', 'new': '1080', 'owner': 
    # <ipywidgets.widgets.widget_selection.ToggleButtons object at 0x7f387bc303c8>, 'type': 'change'}
    #print(change)
    main()   

def slider_1_handle_slider_change(change):
    # Distance from screen in meters
    # {'name': 'value', 'old': 12, 'new': 52, 'owner': 
    # <ipywidgets.widgets.widget_int.IntSlider object at 0x7f388067cbe0>, 'type': 'change'}   
    main()

def slider_2_handle_slider_change(change):
    # Diagonal screen size
    main()

# Observe for changes and then call handle.
# Names is passed, but not used as main() is called
toggle_button.observe(toggle_button_handle_change, names='value')
slider_1.observe(slider_1_handle_slider_change, names='value')
slider_2.observe(slider_2_handle_slider_change, names='value')

#=== Calculation Functions

def get_vertical_pixel(vertical_pixel_string):
    # 0. Provide vertical pixels as string. Return as integer
    return int(vertical_pixel_string)

def get_horizontal_pixel(vertical_pixel):
    # 1. Provide Vertical pixel and return horizontal pixels based on
    # ASPECT_RATIO = (16,9) tuple.
    return int((vertical_pixel / ASPECT_RATIO[1]) * ASPECT_RATIO[0])        

def get_diagonal_in_cm(diagonal_in_inch):
    # 3. Convert diagonal of screen in inch to cm.
    return diagonal_in_inch * CM_PER_INCH

def get_tan_aspect_ratio(aspect_ratio):
    # 4. Calculate the tan of the aspect ratio
    # E.g 16:9 is a = 16 o = 9. tan = o/a
    return aspect_ratio[1] / aspect_ratio[0]

def get_degrees_of_aspect_ratio(tan_aspect_ratio):
    # 5. Angle in degrees of 16:9 aspect ratio = 29.357753542791276°
    return math.degrees(math.atan(tan_aspect_ratio))

def get_atan_of_aspect_ratio(tan_aspect_ratio):
    # 6. Get atan of the tan of the aspect ratio
    return math.atan(tan_aspect_ratio)

def get_screen_width_in_cm(screen_diagonal_in_cm, atan_aspect_ratio):
    # 7. Get screen width in cm
    return math.cos(atan_aspect_ratio) * screen_diagonal_in_cm
    # 18.357559751

def get_screen_height_in_cm(screen_diagonal_in_cm, atan_aspect_ratio):
    # 8.
    return math.sin(atan_aspect_ratio) * screen_diagonal_in_cm

def get_half_screen_width_in_cm(screen_width_in_cm):
    # 9.
    return screen_width_in_cm / 2

def get_screen_distance_in_cm(screen_distance_in_m):
    # 10.
    return screen_distance_in_m * 100 

def get_half_tan_viewing_ratio(half_screen_width_in_cm, screen_distance_in_cm):
    # 11. tan = opp / adj
    return half_screen_width_in_cm / screen_distance_in_cm 

def get_viewing_angle_degrees(half_tan_viewing_ratio):
    # 12. Get the viewing angle. LHS side to RHS side of screen
    half_angle= math.degrees(math.atan(half_tan_viewing_ratio))
    return half_angle * 2    

def get_average_pixel_per_degree(viewing_angle_degrees, horizontal_pixel):
    # 13.
    return horizontal_pixel / viewing_angle_degrees

def get_length_to_side_of_screen_cm(screen_distance_in_cm, half_screen_width_in_cm):
    # 14.
    return math.sqrt(screen_distance_in_cm ** 2 + half_screen_width_in_cm ** 2)

def get_diff_centre_to_side_cm(screen_distance_in_cm, length_to_side_of_screen_cm):
    # 15.
    return length_to_side_of_screen_cm - screen_distance_in_cm

def get_diff_centre_to_side_percent(screen_distance_in_cm, length_to_side_of_screen_cm):
    # 16.
    return (length_to_side_of_screen_cm - screen_distance_in_cm) / screen_distance_in_cm * 100

#=== Main. Executed after every change to a widget

def main():
    # Toggle Button array is via toggle_button.value
    # Slider1 and 2 data is via slider_1.value and slider_2.value
    # 
    # Build data as list
    data = []
    
    # 0. From toggle button get the vertical pixels
    vertical_pixel = get_vertical_pixel(toggle_button.value)
    data.append(vertical_pixel)
    
    # 1. Get the horizontal pixels based on 16:9 ratio
    horizontal_pixel = get_horizontal_pixel(vertical_pixel)
    data.append(horizontal_pixel)
    
    # 2. Screen diagonal as inch
    screen_diagonal_in_inch = slider_2.value
    # Test: 7.227 in = 18.35 cm = width 16cm height 9cm. Angle:29.357753542791276°
    #screen_diagonal_in_inch = 7.227385729 # = 18.357559751 cm
    data.append(screen_diagonal_in_inch)
    
    # 3. Screen diagonal as cm
    screen_diagonal_in_cm = get_diagonal_in_cm(screen_diagonal_in_inch)
    data.append("{0:.1f}".format(screen_diagonal_in_cm)) 

    # 4. Tan of 16:9 aspect ratio 
    tan_aspect_ratio = get_tan_aspect_ratio(ASPECT_RATIO)   
    data.append(tan_aspect_ratio)  # 0.5625  

    # 5. Check the angle of the aspect ratio vert = oppo, horizontal = adjcent
    degrees_aspect_ratio = get_degrees_of_aspect_ratio(tan_aspect_ratio)
    data.append(degrees_aspect_ratio) 
    
    # 6. Get atan of aspect ratio to use in calculating horizontal and vertical
    atan_aspect_ratio = get_atan_of_aspect_ratio(tan_aspect_ratio)
    data.append(atan_aspect_ratio)
    
    # 7. Width of screen in cm
    screen_width_in_cm = get_screen_width_in_cm(screen_diagonal_in_cm, atan_aspect_ratio)
    data.append("{0:.1f}".format(screen_width_in_cm))

    # 8. Height of screen in cm
    screen_height_in_cm = get_screen_height_in_cm(screen_diagonal_in_cm, atan_aspect_ratio)
    data.append("{0:.1f}".format(screen_height_in_cm))
    
    # 9. Half screen_width_in_cm. 
    # Becomes opposite side in viewing angle calculations
    half_screen_width_in_cm = get_half_screen_width_in_cm(screen_width_in_cm)
    data.append(half_screen_width_in_cm)

    # 10. Screen distance in cm. Slider is in meters 
    # Becomes the adjacent side in viewing angle calculations
    screen_distance_in_cm = get_screen_distance_in_cm(slider_1.value)
    data.append("{0:.0f}".format(screen_distance_in_cm))    

    # 11. Half tan viewing ratio
    half_tan_viewing_ratio = get_half_tan_viewing_ratio(half_screen_width_in_cm, screen_distance_in_cm)    
    data.append(half_tan_viewing_ratio)
    
    # 12. Viewing angle
    viewing_angle_degrees = get_viewing_angle_degrees(half_tan_viewing_ratio)
    data.append("{0:.1f}".format(viewing_angle_degrees))

    # 13. Average of pixels per degree
    average_pixel_per_degree = get_average_pixel_per_degree(viewing_angle_degrees, horizontal_pixel)
    data.append("{0:.1f}".format(average_pixel_per_degree))    

    # 14. Length to side of screen
    length_to_side_of_screen_cm = get_length_to_side_of_screen_cm(screen_distance_in_cm, half_screen_width_in_cm)
    data.append("{0:.1f}".format(length_to_side_of_screen_cm)) 
    
    # 15. Difference in cm length to side from length to centre
    diff_centre_to_side_cm = get_diff_centre_to_side_cm(screen_distance_in_cm, length_to_side_of_screen_cm)
    data.append("{0:.1f}".format(diff_centre_to_side_cm))    
    
    # 16. Difference in % length to side from length to centre
    diff_centre_to_side_percent = get_diff_centre_to_side_percent(screen_distance_in_cm, length_to_side_of_screen_cm)
    data.append("{0:.1f}".format(diff_centre_to_side_percent))    
    
    
    # Display data in list    
    # All data for debugging
    #s = ""
    #for index, item in enumerate(data):
    #    s += DATA_DESCRIPTION[index] + ": " + str(item) + DATA_UNIT[index] + "\n"
    #text_area.value=s
    
    # Display only the relevant data
    s = ""
    for value in RELEVANT_DATA:
        s += DATA_DESCRIPTION[value] + ": " + str(data[value]) + DATA_UNIT[value] + "\n"
    text_area.value=s

#=== Call main for initial data and display widgets in a Vertical Box
main()
display(widgets.VBox([toggle_button, slider_1, slider_2, text_area])) 